# Meta - Ad Segment Performance Analysis

```SQL
CREATE TABLE ad_performance (
    ad_id INTEGER,
    audience_segment_id INTEGER,
    impressions INTEGER,
    conversions INTEGER,
    ad_spend DECIMAL,
    date DATE
);

CREATE TABLE audience_segments (
    audience_segment_id INTEGER,
    segment_name VARCHAR
);


INSERT INTO ad_performance (date, ad_id, ad_spend, conversions, impressions, audience_segment_id)
VALUES
    ('2024-10-02', 1, 38.00, 80, 10000, 1),
    ('2024-10-14', 2, 44.25, 90, 10000, 1),
    ('2024-10-02', 3, 20.00, 50, 5000, 2),
    ('2024-10-10', 4, 35.25, 70, 8000, 2),
    ('2024-10-20', 5, 60.50, 110, 12000, 2),
    ('2024-10-10', 6, 40.00, 60, 8000, 3),
    ('2024-10-23', 7, 48.25, 80, 10000, 3),
    ('2024-10-29', 8, 50.00, 115, 11000, 3),
    ('2024-09-25', 9, 20.00, 35, 5000, 1),
    ('2024-09-15', 10, 15.00, 25, 3000, 2),
    ('2024-10-05', 11, 25.00, 40, 6000, 4),
    ('2024-10-20', 12, 30.00, 20, 7000, 8),
    ('2024-08-10', 13, 10.00, 15, 3000, 5),
    ('2024-10-15', 14, 18.75, 30, 4000, 6),
    ('2024-09-10', 15, 25.00, 20, 4500, 7);

INSERT INTO audience_segments (audience_segment_id, segment_name)
VALUES
    (1, 'Custom Audience - App Installers'),
    (2, 'Custom Audience - Email List Subscribers'),
    (3, 'Custom Audience - Video Viewers (25% Watched)'),
    (4, 'Lookalike Audience - Purchasers'),
    (5, 'Lookalike Audience - Website Visitors'),
    (6, 'Interest-based Audience - Fitness Enthusiasts'),
    (7, 'Interest-based Audience - Gamers'),
    (8, 'Saved Audience - US, 18-24'),
    (9, 'Saved Audience - Europe, 25-34');

```

In [3]:
import pandas as pd
import numpy as np

In [4]:
df_performance = pd.read_csv('Data/005/ad_performance.csv')
df_audience = pd.read_csv('Data/005/audience_segments.csv')

df_performance.head()

,ad_id,audience_segment_id,impressions,conversions,ad_spend,date
0,1,1,10000,80,38.00,2024-10-02
1,2,1,10000,90,44.25,2024-10-14
2,3,2,5000,50,20.00,2024-10-02
3,4,2,8000,70,35.25,2024-10-10
4,5,2,12000,110,60.50,2024-10-20


In [5]:
df_audience.head()

,audience_segment_id,segment_name
0,1,Custom Audience - App Installers
1,2,Custom Audience - Email List Subscribers
2,3,Custom Audience - Video Viewers (25% Watched)
3,4,Lookalike Audience - Purchasers
4,5,Lookalike Audience - Website Visitors


# Pregunta 1

### ¿Cuántas impresiones de anuncios en total recibimos de los segmentos de audiencia personalizados (custom audience segments) en octubre de 2024?

In [13]:
df_custom = pd.merge(df_performance, df_audience, on='audience_segment_id')

df_octubre = (df_custom['date'].between('2024-10-01','2024-10-31')) & (df_custom['segment_name'].str.startswith('Custom Audience'))

total_impresions = df_custom.loc[df_octubre, 'impressions'].sum()

total_impresions

np.int64(74000)

```SQL
SELECT
    SUM(p.impressions)
FROM ad_performance p
JOIN audience_segments s ON p.audience_segment_id = s.audience_segment_id
WHERE EXTRACT (YEAR FROM p.date) = 2024
AND EXTRACT (MONTH FROM p.date) = 10
AND s.segment_name like 'Custom Audience%';
```

# Pregunta 2

### ¿Cuál es el número total de conversiones que logramos de cada segmento de audiencia personalizado (custom audience segment) en octubre de 2024?

In [18]:
df_merge2 = pd.merge(df_audience, df_performance, on='audience_segment_id')

df_octubre2 = df_merge2[
    (df_merge2['date'].between('2024-10-01','2024-10-31')) &
    (df_merge2['segment_name'].str.startswith('Custom Audience'))
]

respuesta2 = df_octubre2.groupby('segment_name')['conversions'].sum().reset_index()

respuesta2

,segment_name,conversions
0,Custom Audience - App Installers,170
1,Custom Audience - Email List Subscribers,230
2,Custom Audience - Video Viewers (25% Watched),255


```SQL
SELECT
    s.segment_name,
    SUM(p.conversions)
FROM ad_performance p
JOIN audience_segments s ON p.audience_segment_id = s.audience_segment_id
WHERE p.date BETWEEN '2024-10-01' AND '2024-10-31'
AND s.segment_name LIKE 'Custom Audience%'
GROUP BY s.segment_name;
```

# Pregunta 3

### Para cada segmento de 'custom audience' (audiencia personalizada) o 'lookalike' (audiencia similar), calcula el costo por conversión. Devuelve esto únicamente para los segmentos que tuvieron un gasto (spend) mayor a cero y conversiones mayores a cero.

### Costo por conversión = Gasto publicitario total / Número total de conversiones

In [20]:
# 1. Filtramos los nombres primero
df_filtrado = df_merge2[df_merge2['segment_name'].str.contains('Custom Audience|Lookalike')]

# 2. Agrupamos y sumamos ambos valores
resumen = df_filtrado.groupby('segment_name')[['ad_spend', 'conversions']].sum()

# 3. Filtramos los totales que sean 0 y calculamos
resumen = resumen[(resumen['ad_spend'] > 0) & (resumen['conversions'] > 0)]
resumen['cost_per_conversion'] = resumen['ad_spend'] / resumen['conversions']

resumen

,ad_spend,conversions,cost_per_conversion
segment_name,,,
Custom Audience - App Installers,102.25,205,0.498780
Custom Audience - Email List Subscribers,130.75,255,0.512745
Custom Audience - Video Viewers (25% Watched),138.25,255,0.542157
Lookalike Audience - Purchasers,25.00,40,0.625000
Lookalike Audience - Website Visitors,10.00,15,0.666667


```SQL
SELECT
    s.segment_name,
    SUM(p.ad_spend) / SUM(p.conversions) AS cost_per_conversion
FROM ad_performance p
JOIN audience_segments s ON p.audience_segment_id = s.audience_segment_id
WHERE (s.segment_name LIKE 'Custom Audience%' OR s.segment_name LIKE 'Lookalike%')
GROUP BY s.segment_name
HAVING SUM(p.ad_spend) > 0 AND SUM(p.conversions) > 0;
```